In [1]:
import os, sys
import scipy.io as sio
import warnings
warnings.filterwarnings('ignore')
import time
from joblib import Parallel, delayed

import numpy as np
from sklearn.model_selection import ShuffleSplit,LeaveOneOut
from toolbox1 import PreProcessing_BETA,acc_calculate,TRCA_train,TRCA_test
from toolbox1 import get_P,TDCA_train,TDCA_test
from toolbox1 import get_augment_fb_noiseAfter_ms
warnings.filterwarnings('ignore')

In [2]:
# main function
def BETA_TRCAmssame(idx_num, n_train, t_task, n_Aug, n_Neig=None):

    # setting
    f_list = np.array([8.6, 8.8,
              9, 9.2, 9.4, 9.6, 9.8,
              10, 10.2, 10.4, 10.6, 10.8,
              11, 11.2, 11.4, 11.6, 11.8,
              12, 12.2, 12.4, 12.6, 12.8,
              13, 13.2, 13.4, 13.6, 13.8,
              14, 14.2, 14.4, 14.6, 14.8,
              15, 15.2, 15.4, 15.6, 15.8,
              8, 8.2, 8.4, ])
    target_order = np.argsort(f_list)
    f_list = f_list[target_order]
    phase_list = np.array([
        0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5,
         0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5,
         0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5,
         0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5,
         0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5,
    ])
    subject_id = ['S'+'{:02d}'.format(idx_subject+1) for idx_subject in range(70)]

    idx_num = idx_num
    idx_subject = subject_id[idx_num]
    sfreq = 250
    filepath = os.path.join(filepath_ori, str(idx_subject) + '.mat')
    num_filter = 5
    preEEG = PreProcessing_BETA(filepath, t_begin=0.5, t_end=0.5 + 0.13 + t_task,  # t_begin=0.5+0.14, t_end=0.5+0.14+0.3
                           fs_down=250, chans=['POZ', 'PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'O1', 'OZ', 'O2'],
                           num_filter=num_filter)

    raw_data = preEEG.load_data()
    w_pass_2d = np.array([[5, 14, 22, 30, 38], [90, 90, 90, 90, 90]])  # 70
    w_stop_2d = np.array([[3, 12, 20, 28, 36], [92, 92, 92, 92, 92]])  # 72
    filtered_data = preEEG.filtered_data_iir111(w_pass_2d, w_stop_2d, raw_data)

    filtered_data['bank1'] = filtered_data['bank1'][:, : ,target_order,:]  # Sorted by frequency in ascending order
    filtered_data['bank2'] = filtered_data['bank2'][:, :, target_order, :]
    filtered_data['bank3'] = filtered_data['bank3'][:, :, target_order, :]
    filtered_data['bank4'] = filtered_data['bank4'][:, :, target_order, :]
    filtered_data['bank5'] = filtered_data['bank5'][:, :, target_order, :]

    """
     Cross-validation parameters
    """
    nBlock = 4
    nEvent = 40
    train_size = n_train  # input
    n_splits = 4
    if train_size == nBlock - 1 or train_size == 1:
        kf = LeaveOneOut()
    else:
        kf = ShuffleSplit(n_splits=n_splits, train_size=train_size, random_state=idx_num+1)

    t = t_task              # input
    task_point = np.arange(int((0.13) * sfreq), int((0.13 + t) * sfreq))

    # train
    acc_s = 0
    for train, test in kf.split(np.arange(nBlock)):
        if train_size == 1:
            train, test = test, train
        # train : get ensembleW of banks
        train_w = dict()
        train_meantemp = dict()
        for idx_filter in range(num_filter):
            idx_filter += 1
            bank_data = filtered_data['bank' + str(idx_filter)]
            train_data11 = bank_data[:, :, :, train]
            train_data = train_data11[:, task_point, :, :]  # n_channels * n_times * n_events * n_trials

            if n_Aug == 0:
                trainData_pt = train_data.copy()
            else:
                # Data augmentation
                ntrail_noise = n_Aug
                data_augment = np.zeros((train_data.shape[0], train_data.shape[1], train_data.shape[2], ntrail_noise))
                for ievent in range(nEvent):
                    # get Nh_strat
                    f = f_list[ievent]
                    for ih in range(5):
                        ih = ih + 1
                        if ih * f >= 8 * idx_filter:
                            Nh_start = ih
                            break
                    data_augment[:, :, ievent, :] = get_augment_fb_noiseAfter_ms(fs=sfreq, f_list=f_list,
                                                                                 phi_list=phase_list, Nh_start=Nh_start,
                                                                                 Nh_end=5,
                                                                                 ntrail_noise=ntrail_noise,
                                                                                 mean_temp_all=np.mean(train_data, -1),
                                                                                 iEvent=ievent, nTemplates=n_Neig)
                trainData_pt = np.concatenate((train_data, data_augment), axis=3)

            # train
            w, mean_temp = TRCA_train(trainData_pt)
            train_w['bank' + str(idx_filter)] = w
            train_meantemp['bank' + str(idx_filter)] = mean_temp

        # test:
        predictAll = np.zeros((test.shape[0], nEvent),int)
        flag = 0
        for isplit in test:
            rrall = np.zeros((nEvent, nEvent))
            for idx_filter in range(num_filter):
                idx_filter += 1
                bank_data = filtered_data['bank' + str(idx_filter)]
                test_data = bank_data[:, :, :, isplit]
                test_data = test_data[:, task_point, :]
                rr = TRCA_test(test_data, train_w['bank' + str(idx_filter)],
                               train_meantemp['bank' + str(idx_filter)], True)
                rrall += np.multiply(np.sign(rr), (rr ** 2)) * (idx_filter ** (-1.25) + 0.25)
            predict = np.argmax(rrall, -1)
            predictAll[flag, :] = predict
            flag += 1
        acc_s = acc_calculate(predictAll) + acc_s
    acc = acc_s / n_splits
    print('sub', idx_num + 1, ', acc = ', acc_s / n_splits)
    return acc

def BETA_TDCAmssame(idx_num, n_train, t_task, n_Aug, n_Neig=None):

    # setting
    f_list = np.array([8.6, 8.8,
              9, 9.2, 9.4, 9.6, 9.8,
              10, 10.2, 10.4, 10.6, 10.8,
              11, 11.2, 11.4, 11.6, 11.8,
              12, 12.2, 12.4, 12.6, 12.8,
              13, 13.2, 13.4, 13.6, 13.8,
              14, 14.2, 14.4, 14.6, 14.8,
              15, 15.2, 15.4, 15.6, 15.8,
              8, 8.2, 8.4, ])
    target_order = np.argsort(f_list)
    f_list = f_list[target_order]
    phase_list = np.array([
        0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5,
         0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5,
         0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5,
         0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5,
         0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5,
    ])
    subject_id = ['S'+'{:02d}'.format(idx_subject+1) for idx_subject in range(70)]

    idx_num = idx_num
    idx_subject = subject_id[idx_num]
    sfreq = 250
    filepath = os.path.join(filepath_ori, str(idx_subject) + '.mat')
    num_filter = 5
    preEEG = PreProcessing_BETA(filepath, t_begin=0.5, t_end=0.5 + 0.13 + t_task + 3/sfreq,
                           fs_down=250, chans=['POZ', 'PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'O1', 'OZ', 'O2'],
                           num_filter=num_filter)

    raw_data = preEEG.load_data()
    w_pass_2d = np.array([[5, 14, 22, 30, 38], [90, 90, 90, 90, 90]])
    w_stop_2d = np.array([[3, 12, 20, 28, 36], [92, 92, 92, 92, 92]])
    filtered_data = preEEG.filtered_data_iir111(w_pass_2d, w_stop_2d, raw_data)

    filtered_data['bank1'] = filtered_data['bank1'][:, : ,target_order,:]  # Sorted by frequency in ascending order
    filtered_data['bank2'] = filtered_data['bank2'][:, :, target_order, :]
    filtered_data['bank3'] = filtered_data['bank3'][:, :, target_order, :]
    filtered_data['bank4'] = filtered_data['bank4'][:, :, target_order, :]
    filtered_data['bank5'] = filtered_data['bank5'][:, :, target_order, :]

    """
     Cross-validation parameters
    """
    nBlock = 4
    nEvent = 40
    train_size = n_train   # input
    n_splits = 4
    if train_size == nBlock - 1 or train_size == 1:
        kf = LeaveOneOut()
    else:
        kf = ShuffleSplit(n_splits=n_splits, train_size=train_size, random_state=idx_num+1)

    """
    TDCA parameters
    """
    l = 3  # delay point for TDCA
    sTime = t_task        #   input
    train_point  = np.arange(int((0.13) * sfreq), int((0.13 + sTime) * sfreq)+l)
    test_point = np.arange(int((0.13) * sfreq), int((0.13 + sTime) * sfreq))
    # Obtain the projection matrix P of all classes
    P = get_P(f_list=f_list, Nh=5, sTime=sTime, sfreq=sfreq)

    # train4
    acc_s = 0
    for train, test in kf.split(np.arange(nBlock)):
        if train_size == 1:
            train, test = test, train
        # train : get ensembleW of banks
        train_w = dict()
        train_meantemp = dict()
        for idx_filter in range(num_filter):
            idx_filter += 1
            bank_data = filtered_data['bank' + str(idx_filter)]
            train_data11 = bank_data[:, :, :, train]
            train_data = train_data11[:, train_point, :, :]  # n_channels * n_times * n_events * n_trials

            if n_Aug == 0:
                trainData_pt = train_data.copy()
            else:
                # Data augmentation
                ntrail_noise = n_Aug
                data_augment = np.zeros((train_data.shape[0], train_data.shape[1], train_data.shape[2], ntrail_noise))
                for ievent in range(nEvent):
                    # get Nh_strat
                    f = f_list[ievent]
                    for ih in range(5):
                        ih = ih + 1
                        if ih * f >= 8 * idx_filter:
                            Nh_start = ih
                            break
                    data_augment[:, :, ievent, :] = get_augment_fb_noiseAfter_ms(fs=sfreq, f_list=f_list,
                                                                                 phi_list=phase_list, Nh_start=Nh_start,
                                                                                 Nh_end=5,
                                                                                 ntrail_noise=ntrail_noise,
                                                                                 mean_temp_all=np.mean(train_data, -1),
                                                                                 iEvent=ievent, nTemplates=n_Neig)
                trainData_pt = np.concatenate((train_data, data_augment), axis=3)

            # train
            w, mean_temp_TDCA = TDCA_train(trainData_pt, P=P, l=l, Nk=9)
            train_w['bank' + str(idx_filter)] = w
            train_meantemp['bank' + str(idx_filter)] = mean_temp_TDCA

        # test:
        predictAll = np.zeros((test.shape[0], nEvent),int)
        flag = 0
        for isplit in test:
            rrall = np.zeros((nEvent, nEvent))
            for idx_filter in range(num_filter):
                idx_filter += 1
                bank_data = filtered_data['bank' + str(idx_filter)]
                test_data111 = bank_data[:, :, :, isplit]
                test_data = test_data111[:,test_point,:]
                rr = TDCA_test(test_data, train_w['bank' + str(idx_filter)], train_meantemp['bank' + str(idx_filter)],
                               P=P, l=l)
                rrall += np.multiply(np.sign(rr), (rr ** 2)) * (idx_filter ** (-1.25) + 0.25)
            predict = np.argmax(rrall, -1)
            predictAll[flag, :] = predict
            flag += 1
        acc_s = acc_calculate(predictAll) + acc_s
    acc = acc_s / n_splits
    print('sub', idx_num + 1, ', acc = ', acc_s / n_splits)
    return acc

### main

In [3]:
# setting
n_subjects = 70
Train_size_list = [1,2,3]
nAug_list=[3,5,6]
t_task_list = [0.5]
# nNeig_list = [i*2 for i in range(21)]
filepath_ori = '/mnt/Beta'

### eTRCA

In [4]:
######## eTRCA withoutSAME ########
print('eTRCA withoutSAME is executing...')
# acc = BETA_TRCAmssame(idx_num=57, n_train = 2 ,t_task=0.5, n_Aug=0)
acc_all = np.zeros((n_subjects,len(Train_size_list),len(t_task_list)))
for i, i_times in enumerate(t_task_list):
    for j ,j_train in enumerate(Train_size_list):
        T1 = time.time()
        acc = Parallel(n_jobs=-1)(delayed(BETA_TRCAmssame)(idx_num, n_train=j_train, t_task=i_times,n_Aug=0) for idx_num in range(n_subjects))
        acc = np.array(acc)
        acc_all[:,j,i] = acc
        T2 = time.time()
        print('n_times=',i_times,';n_trian=',j_train,';mean_acc=',np.mean(acc),'; running time=',T2 - T1)
# sio.savemat(r'BETA_eTRCA_acc.mat', {'acc': acc_all})

eTRCA withoutSAME is executing...
n_times= 0.5 ;n_trian= 1 ;mean_acc= 0.12014880952380952 ; running time= 102.4008948802948
n_times= 0.5 ;n_trian= 2 ;mean_acc= 0.5208035714285714 ; running time= 81.31394457817078
n_times= 0.5 ;n_trian= 3 ;mean_acc= 0.609017857142857 ; running time= 54.879578828811646


### eTRCA(w/SAME)

In [5]:
######## eTRCA with SAME ########
print('eTRCA with SAME is executing...')
# acc = BETA_TRCAmssame(idx_num=57, n_train = 2 ,t_task=0.5, n_Aug=nAug_list[1],n_Neig=0)
acc_all = np.zeros((n_subjects,len(Train_size_list),len(t_task_list)))
for i, i_times in enumerate(t_task_list):
    for j ,j_train in enumerate(Train_size_list):
        T1 = time.time()
        acc = Parallel(n_jobs=-1)(delayed(BETA_TRCAmssame)(idx_num, n_train=j_train, t_task=i_times,n_Aug=nAug_list[j],n_Neig=0) for idx_num in range(n_subjects))
        acc = np.array(acc)
        acc_all[:,j,i] = acc
        T2 = time.time()
        print('n_times=',i_times,';n_trian=',j_train,';mean_acc=',np.mean(acc),'; running time=',T2 - T1)
# sio.savemat(r'BETA_eTRCA_SAME_acc.mat', {'acc': acc_all})

eTRCA with SAME is executing...
n_times= 0.5 ;n_trian= 1 ;mean_acc= 0.5208630952380952 ; running time= 109.84040117263794
n_times= 0.5 ;n_trian= 2 ;mean_acc= 0.6249999999999999 ; running time= 88.12679767608643
n_times= 0.5 ;n_trian= 3 ;mean_acc= 0.6776785714285716 ; running time= 68.85598540306091


### eTRCA(w/msSAME)

In [6]:
######## eTRCA with msSAME ########
print('eTRCA with msSAME is executing...')
# acc = BETA_TRCAmssame(idx_num=57, n_train = 2 ,t_task=0.5, n_Aug=nAug_list[1],n_Neig=10)
acc_all = np.zeros((n_subjects,len(Train_size_list),len(t_task_list)))
for i, i_times in enumerate(t_task_list):
    for j ,j_train in enumerate(Train_size_list):
        T1 = time.time()
        acc = Parallel(n_jobs=-1)(delayed(BETA_TRCAmssame)(idx_num, n_train=j_train, t_task=i_times,n_Aug=nAug_list[j],n_Neig=10) for idx_num in range(n_subjects))
        acc = np.array(acc)
        acc_all[:,j,i] = acc
        T2 = time.time()
        print('n_times=',i_times,';n_trian=',j_train,';mean_acc=',np.mean(acc),'; running time=',T2 - T1)
# sio.savemat(r'BETA_eTRCA_msSAME_acc.mat', {'acc': acc_all})

eTRCA with msSAME is executing...
n_times= 0.5 ;n_trian= 1 ;mean_acc= 0.6247916666666667 ; running time= 113.67107391357422
n_times= 0.5 ;n_trian= 2 ;mean_acc= 0.6879017857142858 ; running time= 95.79490780830383
n_times= 0.5 ;n_trian= 3 ;mean_acc= 0.7124107142857142 ; running time= 76.81982493400574


### TDCA

In [7]:
######## TDCA withoutSAME ########
print('TDCA withoutSAME is executing...')
# acc = BETA_TDCAmssame(idx_num=57, n_train = 2 ,t_task=0.5, n_Aug=0)
acc_all = np.zeros((n_subjects,len(Train_size_list),len(t_task_list)))
for i, i_times in enumerate(t_task_list):
    for j ,j_train in enumerate(Train_size_list):
        T1 = time.time()
        acc = Parallel(n_jobs=-1)(delayed(BETA_TDCAmssame)(idx_num, n_train=j_train, t_task=i_times,n_Aug=0) for idx_num in range(n_subjects))
        acc = np.array(acc)
        acc_all[:,j,i] = acc
        T2 = time.time()
        print('n_times=',i_times,';n_trian=',j_train,';mean_acc=',np.mean(acc),'; running time=',T2 - T1)
# sio.savemat(r'BETA_TDCA_acc.mat', {'acc': acc_all})

TDCA withoutSAME is executing...
n_times= 0.5 ;n_trian= 1 ;mean_acc= 0.18431547619047617 ; running time= 923.5383055210114
n_times= 0.5 ;n_trian= 2 ;mean_acc= 0.6082589285714286 ; running time= 643.8419704437256
n_times= 0.5 ;n_trian= 3 ;mean_acc= 0.6830357142857142 ; running time= 354.77954363822937


### TDCA(w/SAME)

In [8]:
######## TDCA with SAME ########
print('TDCA with SAME is executing...')
# acc = BETA_TDCAmssame(idx_num=57, n_train = 2 ,t_task=0.5, n_Aug=nAug_list[1],n_Neig=0)
acc_all = np.zeros((n_subjects,len(Train_size_list),len(t_task_list)))
for i, i_times in enumerate(t_task_list):
    for j ,j_train in enumerate(Train_size_list):
        T1 = time.time()
        acc = Parallel(n_jobs=-1)(delayed(BETA_TDCAmssame)(idx_num, n_train=j_train, t_task=i_times,n_Aug=nAug_list[j],n_Neig=0) for idx_num in range(n_subjects))
        acc = np.array(acc)
        acc_all[:,j,i] = acc
        T2 = time.time()
        print('n_times=',i_times,';n_trian=',j_train,';mean_acc=',np.mean(acc),'; running time=',T2 - T1)
# sio.savemat(r'BETA_TDCA_SAME_acc.mat', {'acc': acc_all})

TDCA with SAME is executing...
n_times= 0.5 ;n_trian= 1 ;mean_acc= 0.5263690476190476 ; running time= 962.561327457428
n_times= 0.5 ;n_trian= 2 ;mean_acc= 0.6345535714285714 ; running time= 698.0998861789703
n_times= 0.5 ;n_trian= 3 ;mean_acc= 0.695982142857143 ; running time= 423.4984850883484


### TDCA(w/msSAME)

In [9]:
######## TDCA with msSAME ########
print('TDCA with msSAME is executing...')
# acc = BETA_TDCAmssame(idx_num=57, n_train = 2 ,t_task=0.5, n_Aug=nAug_list[1],n_Neig=10)
acc_all = np.zeros((n_subjects,len(Train_size_list),len(t_task_list)))
for i, i_times in enumerate(t_task_list):
    for j ,j_train in enumerate(Train_size_list):
        T1 = time.time()
        acc = Parallel(n_jobs=-1)(delayed(BETA_TDCAmssame)(idx_num, n_train=j_train, t_task=i_times,n_Aug=nAug_list[j],n_Neig=10) for idx_num in range(n_subjects))
        acc = np.array(acc)
        acc_all[:,j,i] = acc
        T2 = time.time()
        print('n_times=',i_times,';n_trian=',j_train,';mean_acc=',np.mean(acc),'; running time=',T2 - T1)
# sio.savemat(r'BETA_TDCA_msSAME_acc.mat', {'acc': acc_all})

TDCA with msSAME is executing...
n_times= 0.5 ;n_trian= 1 ;mean_acc= 0.6719642857142857 ; running time= 962.2310721874237
n_times= 0.5 ;n_trian= 2 ;mean_acc= 0.7314732142857144 ; running time= 697.8803493976593
n_times= 0.5 ;n_trian= 3 ;mean_acc= 0.7577678571428572 ; running time= 423.3291039466858
